In [ ]:
# Importar bibliotecas necesarias
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import math
import yfinance as yf
import matplotlib.pyplot as plt

# Definición de la clase para el modelo Black-Scholes (Incluyendo las funciones de Rho y vs T)
class BlackScholesModel:
    r"""
    Implementa el modelo Black-Scholes-Merton (BSM) para calcular el
    precio teórico de opciones financieras (Call y Put) y sus Griegas.

    Parámetros de inicialización:
    S (float): Precio actual de la acción (Underlying Price).
    K (float): Precio de ejercicio (Strike Price).
    T (float): Tiempo hasta el vencimiento (en años, e.g., 0.5 para 6 meses).
    r (float): Tasa de interés libre de riesgo (anualizada).
    sigma (float): Volatilidad anualizada del activo subyacente.
    """
    def __init__(self, S, K, T, r, sigma):
        # Almacenar los parámetros de la opción
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma

        # Calcular d1 y d2 al inicializar
        self.d1, self.d2 = self._calculate_d1_d2()

    def _calculate_d1_d2(self):
        r"""
        Calcula los parámetros d1 y d2 del modelo Black-Scholes.
        """
        # Evitar la división por cero si T es muy pequeño o sigma es cero
        if self.T <= 0 or self.sigma <= 0:
            if self.T <= 0:
                 return 0.0, 0.0

        # d1:
        d1 = (np.log(self.S / self.K) + (self.r + self.sigma**2 / 2) * self.T) / (self.sigma * np.sqrt(self.T))

        # d2:
        d2 = d1 - self.sigma * np.sqrt(self.T)

        return d1, d2

    # --- Funciones de Precios (Value) ---

    def call_price(self):
        r"""
        Calcula el precio teórico de una opción Call.
        """
        if self.T <= 0 or self.sigma <= 0:
            return max(0, self.S - self.K)

        d1, d2 = self.d1, self.d2

        # N(d1) y N(d2) son las funciones de distribución acumulada normal estándar
        C = self.S * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        return C

    def put_price(self):
        r"""
        Calcula el precio teórico de una opción Put.
        """
        if self.T <= 0 or self.sigma <= 0:
            return max(0, self.K - self.S)

        d1, d2 = self.d1, self.d2

        # N(-d1) y N(-d2)
        P = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S * norm.cdf(-d1)
        return P

    # --- Funciones de Griegas (Greeks) ---

    def _pdf(self, d):
        return norm.pdf(d)

    def delta(self, option_type='call'):
        r"""
        Calcula el Delta ($\Delta$): sensibilidad del precio de la opción a cambios en el precio
        del activo subyacente (S).
        """
        d1 = self.d1
        if option_type.lower() == 'call':
            return norm.cdf(d1)
        elif option_type.lower() == 'put':
            return norm.cdf(d1) - 1.0
        return 0.0

    def gamma(self):
        r"""
        Calcula el Gamma ($\Gamma$): sensibilidad del Delta a cambios en el precio
        del activo subyacente (S).
        (Es el mismo para Calls y Puts).
        """
        d1 = self.d1
        if self.T <= 0 or self.sigma <= 0 or self.S <= 0:
            return 0.0
        return self._pdf(d1) / (self.S * self.sigma * np.sqrt(self.T))

    def theta(self, option_type='call'):
        r"""
        Calcula el Theta ($\Theta$): sensibilidad del precio de la opción al paso del tiempo (T).
        (Se calcula como el cambio de precio por día).
        """
        d1, d2 = self.d1, self.d2

        if self.T <= 0 or self.sigma <= 0:
            return 0.0

        term1 = - (self.S * self._pdf(d1) * self.sigma) / (2 * np.sqrt(self.T))

        if option_type.lower() == 'call':
            term2 = - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2)
        elif option_type.lower() == 'put':
            term2 = + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2)
        else:
            return 0.0

        return (term1 + term2) / 365.0

    def vega(self):
        r"""
        Calcula el Vega ($\mathcal{V}$): sensibilidad del precio de la opción a cambios en la volatilidad (sigma).
        (Es el mismo para Calls y Puts).
        """
        d1 = self.d1
        if self.T <= 0:
            return 0.0

        return self.S * self._pdf(d1) * np.sqrt(self.T)

    def rho(self, option_type='call'):
        r"""
        Calcula el Rho ($\rho$): sensibilidad del precio de la opción a cambios en la tasa
        de interés libre de riesgo (r).
        """
        d2 = self.d2

        if option_type.lower() == 'call':
            return self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2)
        elif option_type.lower() == 'put':
            return -self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2)
        return 0.0

    # --- Funciones para Griegas vs Tiempo (T) ---

    def delta_vs_T(self, T_range, option_type='call'):
        """ Calcula el Delta a través de un rango de T. """
        return [BlackScholesModel(self.S, self.K, t, self.r, self.sigma).delta(option_type) for t in T_range if t > 0]

    def gamma_vs_T(self, T_range):
        """ Calcula el Gamma a través de un rango de T. """
        return [BlackScholesModel(self.S, self.K, t, self.r, self.sigma).gamma() for t in T_range if t > 0]

    def theta_vs_T(self, T_range, option_type='call'):
        """ Calcula el Theta a través de un rango de T. """
        return [BlackScholesModel(self.S, self.K, t, self.r, self.sigma).theta(option_type) for t in T_range if t > 0]

    def vega_vs_T(self, T_range):
        """ Calcula el Vega a través de un rango de T. """
        return [BlackScholesModel(self.S, self.K, t, self.r, self.sigma).vega() for t in T_range if t > 0]

    def rho_vs_T(self, T_range, option_type='call'):
        """ Calcula el Rho a través de un rango de T. """
        return [BlackScholesModel(self.S, self.K, t, self.r, self.sigma).rho(option_type) for t in T_range if t > 0]


# Implementación del modelo Binomial
class BinomialModel:
    r"""
    Implementa el modelo Binomial para calcular el precio teórico de opciones
    financieras (Call y Put).

    Parámetros de inicialización:
    S (float): Precio actual de la acción (Underlying Price).
    K (float): Precio de ejercicio (Strike Price).
    T (float): Tiempo hasta el vencimiento (en años).
    r (float): Tasa de interés libre de riesgo (anualizada).
    sigma (float): Volatilidad anualizada del activo subyacente.
    n (int): Número de pasos en el árbol binomial.
    """
    def __init__(self, S, K, T, r, sigma, n):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.n = int(n) # Asegurar que n sea un entero

        # Calcular parámetros del árbol binomial
        self.dt = self.T / self.n
        self.u = np.exp(self.sigma * np.sqrt(self.dt))
        self.d = 1 / self.u
        self.p = (np.exp(self.r * self.dt) - self.d) / (self.u - self.d) # Probabilidad neutral al riesgo

        # Validar parámetros
        if self.p < 0 or self.p > 1:
            print("Advertencia: La probabilidad neutral al riesgo (p) está fuera del rango [0, 1]. El modelo puede ser inestable.")


    def call_price(self):
        r"""
        Calcula el precio teórico de una opción Call usando el modelo Binomial.
        """
        s_n = np.array([self.S * (self.u**j) * (self.d**(self.n - j)) for j in range(self.n + 1)])
        option_values = np.maximum(0, s_n - self.K)
        for i in range(self.n - 1, -1, -1):
            option_values_at_step_i = np.array([
                np.exp(-self.r * self.dt) * (self.p * option_values[j + 1] + (1 - self.p) * option_values[j])
                for j in range(i + 1)
            ])
            option_values = option_values_at_step_i
        return option_values[0]

    def put_price(self):
        r"""
        Calcula el precio teórico de una opción Put usando el modelo Binomial.
        """
        s_n = np.array([self.S * (self.u**j) * (self.d**(self.n - j)) for j in range(self.n + 1)])
        option_values = np.maximum(0, self.K - s_n)
        for i in range(self.n - 1, -1, -1):
             option_values_at_step_i = np.array([
                np.exp(-self.r * self.dt) * (self.p * option_values[j + 1] + (1 - self.p) * option_values[j])
                for j in range(i + 1)
            ])
             option_values = option_values_at_step_i
        return option_values[0]


# --- Función para Calcular Volatilidad Implícita (IV) ---
def calculate_implied_volatility(market_price, S, K, T, r, option_type='call'):
    r"""
    Calcula la Volatilidad Implícita (IV) usando el método Brentq.
    Encuentra el valor de sigma que iguala el precio teórico BSM al precio de mercado.
    """
    # Precio intrínseco mínimo de la opción
    if option_type.lower() == 'call':
        min_price = max(0, S - K * np.exp(-r * T))
    else:
        min_price = max(0, K * np.exp(-r * T) - S)

    if market_price < min_price:
         # print(f"Advertencia IV: El precio de mercado (${market_price:.4f}) es menor que el valor intrínseco mínimo (${min_price:.4f}).")
         return np.nan # Usar NaN para indicar un precio inválido/inconsistente

    # Función de diferencia (precio de BSM - precio de mercado)
    def difference(sigma):
        # BlackScholesModel requiere sigma > 0.
        # Usamos 1e-6 para evitar sigma cero, aunque brentq maneja el límite.
        if sigma <= 0: return market_price
        model = BlackScholesModel(S, K, T, r, sigma)

        if option_type.lower() == 'call':
            price = model.call_price()
        else: # 'put'
            price = model.put_price()

        return price - market_price

    # Usar brentq para encontrar la raíz (sigma) en un rango plausible (0.0001 a 5.0)
    try:
        # 1e-6 (0.0001%) es el límite inferior de sigma; 5.0 (500%) es el límite superior.
        iv = brentq(difference, 1e-6, 5.0)
        return iv
    except ValueError:
        # Esto ocurre si el precio de mercado está fuera de los límites (demasiado alto o demasiado bajo)
        return np.nan

# --- Función para Calcular Volatilidad Histórica ---
def calculate_historical_volatility(ticker, period="1y", annualize_factor=252):
    r"""
    Calcula la volatilidad histórica anualizada para un ticker y período dados.

    Parámetros:
    ticker (str): Símbolo del activo (ej. "AAPL").
    period (str): Período para obtener datos históricos (ej. "1d", "5d", "1mo", "3mo", "6mo", "1y", "2y", "5y", "10y", "ytd", "max").
    annualize_factor (int): Factor para anualizar la volatilidad (252 para días de trading).

    Retorna:
    float: Volatilidad histórica anualizada.
    """
    try:
        ticker_yf = yf.Ticker(ticker)
        historical_data = ticker_yf.history(period=period)
        historical_prices = historical_data['Close']
        daily_returns = np.log(historical_prices / historical_prices.shift(1)).dropna()
        daily_std_dev = daily_returns.std()
        historical_vol = daily_std_dev * np.sqrt(annualize_factor)
        return historical_vol
    except Exception as e:
        print(f"Error al calcular volatilidad histórica para {ticker} ({period}): {e}")
        return np.nan # Retornar NaN si hay un error


# --- Ejemplo de Uso y Comparación de Modelos ---
if __name__ == "__main__":
    print("---------------------------------------------------------")
    print("   Cálculo del Precio de Opciones y Griegas (Black-Scholes) y Comparación con Modelo Binomial")
    print("---------------------------------------------------------")

    # Parámetros de referencia para la opción
    K_strike = 105.0 # Precio de ejercicio
    T_maturity = 0.5  # Tiempo hasta el vencimiento (0.5 años = 6 meses)
    r_rate = 0.05    # Tasa libre de riesgo (5% anual)
    n_steps_binomial = 500   # Número de pasos para el modelo binomial (para comparación)

    # ---------------------------------------------------------
    # --- Obtener Datos de Mercado y Calcular Volatilidad Histórica ---
    # ---------------------------------------------------------

    TICKER = "AAPL"
    EXPIRY_DATE = "2025-12-19" # Fecha de vencimiento para buscar en yfinance

    current_S = 100.0 # Valor por defecto para S

    try:
        print(f"\n--- Obtención de Datos de Mercado y Volatilidad para {TICKER} ---")

        ticker_yf = yf.Ticker(TICKER)

        # Obtener el precio spot actual
        current_S = ticker_yf.history(period="1d")['Close'].iloc[-1]

        # Calcular Volatilidad Histórica (Con período configurable)
        historical_volatility_period = "1y" # Puedes cambiar el período aquí (ej. "3mo", "6mo", "1y")
        sigma_vol = calculate_historical_volatility(TICKER, period=historical_volatility_period)

        if np.isnan(sigma_vol):
            print(f"Advertencia: No se pudo calcular la volatilidad histórica para el período {historical_volatility_period}. Usando volatilidad por defecto (20%).")
            sigma_vol = 0.20 # Usar volatilidad por defecto si el cálculo falla

        print(f"  S Actual de {TICKER}: ${current_S:.2f}")
        print(f"  Volatilidad Histórica Anualizada ({historical_volatility_period}): {sigma_vol*100:.4f}%")


    except Exception as e:
        print(f"\nError al obtener datos de yfinance o calcular volatilidad: {e}")
        # Asegurarse de que las variables estén definidas incluso si hay un error
        current_S = 100.0 # Usar el S inicial si falla yfinance
        sigma_vol = 0.20 # Usar la volatilidad inicial si yfinance falla
        print("Usando valores por defecto: S=100.0, sigma=0.20")


    # ---------------------------------------------------------
    # --- 3. Comparación con Precios Reales (usando yfinance) ---
    # ---------------------------------------------------------
    # Esta sección ahora usa current_S y sigma_vol obtenidos arriba

    print(f"\n--- Análisis de Mercado para {TICKER} (Strike={K_strike:.2f}, Vencimiento={EXPIRY_DATE}) ---")

    if EXPIRY_DATE not in yf.Ticker(TICKER).options: # Check again if expiry is available
        print(f"Error: La fecha de vencimiento {EXPIRY_DATE} no está disponible para {TICKER}. Saltando comparación de mercado y IV.")
        call_mid_price = np.nan
        put_mid_price = np.nan
        iv_df_clean = None # No hay datos de IV Skew si no hay fecha de vencimiento

    else:
        try:
            ticker_yf = yf.Ticker(TICKER) # Re-instantiate in case of previous error
            option_chain = ticker_yf.option_chain(EXPIRY_DATE)
            calls_df = option_chain.calls
            puts_df = option_chain.puts

            # Buscar el strike K_strike
            call_data = calls_df[calls_df['strike'] == K_strike]
            put_data = puts_df[puts_df['strike'] == K_strike]


            if not call_data.empty and not put_data.empty:
                # Usamos (bid + ask) / 2 como precio de mercado (midPrice)
                call_mid_price = (call_data['bid'].values[0] + call_data['ask'].values[0]) / 2.0
                put_mid_price = (put_data['bid'].values[0] + put_data['ask'].values[0]) / 2.0

                # Recalcular el precio teórico BSM usando el S actual del mercado Y la volatilidad histórica
                bsm_market = BlackScholesModel(
                    S=current_S,
                    K=K_strike,
                    T=T_maturity,
                    r=r_rate,
                    sigma=sigma_vol # <<< Usando sigma_vol (volatilidad histórica)
                )
                call_p_actual = bsm_market.call_price()
                put_p_actual = bsm_market.put_price()

                print(f"  Strike y Vencimiento: K=${K_strike:.2f}, T={T_maturity} años")

                # --- Comparación de Precios y Desviación ---
                call_diff = call_mid_price - call_p_actual
                put_diff = put_mid_price - put_p_actual

                print(f"\n--- Comparación de Precios (usando Volatilidad Histórica de {sigma_vol*100:.2f}%) ---")
                print(f"| Tipo | Mid Price | Teórico | Desviación |")
                print(f"|------|-----------|---------|------------|")
                print(f"| Call | ${call_mid_price:.4f} | ${call_p_actual:.4f} | {call_diff:.4f} |")
                print(f"| Put  | ${put_mid_price:.4f} | ${put_p_actual:.4f} | {put_diff:.4f} |")

                # --- Cálculo de Volatilidad Implícita (IV) ---
                # La IV se calcula a partir del precio de mercado, no usa sigma_vol directamente
                iv_call = calculate_implied_volatility(call_mid_price, current_S, K_strike, T_maturity, r_rate, 'call')
                iv_put = calculate_implied_volatility(put_mid_price, current_S, K_strike, T_maturity, r_rate, 'put')

                print("\n--- Volatilidad Implícita (IV) (La 'sigma' que usa el mercado) ---")
                # Mostrar solo si no son NaN
                if not np.isnan(iv_call):
                    print(f"IV Call: {iv_call*100:.4f}%")
                if not np.isnan(iv_put):
                    print(f"IV Put: {iv_put*100:.4f}%")

                # 6.1. Obtener Strikes y Precios de Mercado para IV Skew
                # Unimos Call y Put dataframes y limpiamos datos
                options_df = calls_df.merge(puts_df, on='strike', suffixes=('_call', '_put'))
                options_df = options_df.dropna(subset=['bid_call', 'ask_call', 'bid_put', 'ask_put'])

                # Calculamos el precio medio de mercado (midPrice)
                options_df['mid_price_call'] = (options_df['bid_call'] + options_df['ask_call']) / 2.0
                options_df['mid_price_put'] = (options_df['bid_put'] + options_df['ask_put']) / 2.0

                # Filtramos opciones con Mid Price cero o volumen/interés abierto muy bajo (opcional, para mayor limpieza)
                options_df = options_df[
                    (options_df['mid_price_call'] > 0.001) &
                    (options_df['mid_price_put'] > 0.001)
                ]

                # 6.2. Calcular IV para todos los Strikes

                iv_results = []
                for index, row in options_df.iterrows():
                    strike = row['strike']

                    # Calcular IV Call
                    iv_call_skew = calculate_implied_volatility(
                        market_price=row['mid_price_call'],
                        S=current_S, K=strike, T=T_maturity, r=r_rate, option_type='call'
                    )

                    # Calcular IV Put
                    iv_put_skew = calculate_implied_volatility(
                        market_price=row['mid_price_put'],
                        S=current_S, K=strike, T=T_maturity, r=r_rate, option_type='put'
                    )

                    iv_results.append({
                        'strike': strike,
                        'iv_call': iv_call_skew * 100 if not np.isnan(iv_call_skew) else np.nan,
                        'iv_put': iv_put_skew * 100 if not np.isnan(iv_put_skew) else np.nan
                    })

                iv_df = options_df.merge(
                    (
                        options_df[['strike']].assign(
                            iv_call=[r['iv_call'] for r in iv_results],
                            iv_put=[r['iv_put'] for r in iv_results]
                        )
                    ),
                    on='strike', how='left'
                )

                # Limpiamos resultados NaN para graficar
                iv_df_clean = iv_df.dropna(subset=['iv_call', 'iv_put'])


            else:
                print(f"Error: No se encontraron datos para el Strike ${K_strike:.2f} en la fecha {EXPIRY_DATE}.")
                call_mid_price = np.nan
                put_mid_price = np.nan
                iv_df_clean = None


        except Exception as e:
            print(f"\nError al obtener datos de opciones de yfinance o calcular IV/comparar: {e}")
            # Asegurarse de que las variables estén definidas incluso if hay un error
            call_mid_price = np.nan
            put_mid_price = np.nan
            iv_df_clean = None


    # ---------------------------------------------------------
    # --- 3.1. Comparar precios teóricos (BSM vs Binomial) para un punto específico ---
    # ---------------------------------------------------------
    # Usamos los parámetros obtenidos de yfinance (current_S, sigma_vol)
    print("\n--- Comparando Precios Teóricos (BSM vs Binomial - Punto Específico) ---")

    bsm_model_point = BlackScholesModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol)
    binomial_model_point = BinomialModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol, n=n_steps_binomial)

    bsm_call_price_point = bsm_model_point.call_price()
    bsm_put_price_point = bsm_model_point.put_price()
    binomial_call_price_point = binomial_model_point.call_price()
    binomial_put_price_point = binomial_model_point.put_price()

    print(f"Parámetros: S={current_S:.2f}, K={K_strike:.2f}, T={T_maturity}, r={r_rate:.4f}, sigma={sigma_vol:.4f}, n={n_steps_binomial}")
    print(f"Precio Teórico Call (Black-Scholes): ${bsm_call_price_point:.4f}")
    print(f"Precio Teórico Put (Black-Scholes):  ${bsm_put_price_point:.4f}")
    print(f"Precio Teórico Call (Binomial - n={n_steps_binomial}): ${binomial_call_price_point:.4f}")
    print(f"Precio Teórico Put (Binomial - n={n_steps_binomial}):  ${binomial_put_price_point:.4f}")
    print(f"Diferencia Absoluta Call (BSM vs Binomial): ${abs(bsm_call_price_point - binomial_call_price_point):.6f}")
    print(f"Diferencia Absoluta Put (BSM vs Binomial):  ${abs(binomial_put_price_point - bsm_put_price_point):.6f}")


    # ---------------------------------------------------------
    # --- 4. Graficar la Sensibilidad (Precio vs S) ---
    # ---------------------------------------------------------

    print("\n--- Generando Gráfico de Sensibilidad (Precio vs Subyacente) ---")

    # Rango de precios del subyacente para graficar
    S_range = np.linspace(current_S * 0.7, current_S * 1.3, 100) # Rango basado en S actual

    # Calcular precios de Call y Put a través del rango S_range, usando sigma_vol
    bsm_call_prices_S = [
        BlackScholesModel(S=s, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol).call_price()
        for s in S_range
    ]
    bsm_put_prices_S = [
        BlackScholesModel(S=s, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol).put_price()
        for s in S_range
    ]
    binomial_call_prices_S = [
        BinomialModel(S=s, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol, n=n_steps_binomial).call_price()
        for s in S_range
    ]
    binomial_put_prices_S = [
        BinomialModel(S=s, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol, n=n_steps_binomial).put_price()
        for s in S_range
    ]


    plt.figure(figsize=(12, 7))

    # Gráfico de Call
    plt.plot(S_range, bsm_call_prices_S, label='Black-Scholes Call', color='blue', linestyle='-', linewidth=2)
    plt.plot(S_range, binomial_call_prices_S, label=f'Binomial Call (n={n_steps_binomial})', color='cyan', linestyle='--', linewidth=2)

    # Gráfico de Put
    plt.plot(S_range, bsm_put_prices_S, label='Black-Scholes Put', color='red', linestyle='-', linewidth=2)
    plt.plot(S_range, binomial_put_prices_S, label=f'Binomial Put (n={n_steps_binomial})', color='orange', linestyle='--', linewidth=2)

    # Líneas de referencia
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.5, label=f'Strike (K={K_strike:.2f})')
    plt.axvline(x=current_S, color='green', linestyle=':', linewidth=1.5, label=f'Spot Price (S={current_S:.2f})')

    plt.xlabel('Precio Subyacente (S)', fontsize=12)
    plt.ylabel('Precio de la Opción', fontsize=12)
    plt.title(r'Comparación de Precios de Opciones (BSM vs Binomial) vs. Precio Subyacente' + f'\n(T={T_maturity} años, r={r_rate:.2%}, $\sigma$={sigma_vol*100:.2f}%)', fontsize=14, fontweight='bold')
    plt.legend(loc='upper left')
    plt.grid(True, linestyle='--')
    plt.show()

    # ---------------------------------------------------------
    # --- 5. Graficar Griegas (Delta, Gamma, Theta, Vega, Rho) vs S ---
    # ---------------------------------------------------------

    print("\n--- Generando Gráficos de Griegas (Delta, Gamma, Theta, Vega, Rho vs Subyacente) ---")

    # Rango de precios del subyacente para graficar
    S_range_greeks = np.linspace(current_S * 0.7, current_S * 1.3, 100) # Rango basado en S actual

    # Calcular Griegas a través del rango S_range_greeks, usando sigma_vol
    call_delta_range = []
    put_delta_range = []
    vega_range = []
    gamma_range = []
    call_theta_range = []
    put_theta_range = []
    call_rho_range = []
    put_rho_range = []


    for s in S_range_greeks:
        model = BlackScholesModel(S=s, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol) # <<< Usando sigma_vol
        call_delta_range.append(model.delta(option_type='call'))
        put_delta_range.append(model.delta(option_type='put'))
        vega_range.append(model.vega())
        gamma_range.append(model.gamma())
        call_theta_range.append(model.theta(option_type='call'))
        put_theta_range.append(model.theta(option_type='put'))
        call_rho_range.append(model.rho(option_type='call'))
        put_rho_range.append(model.rho(option_type='put'))

    # Tamaño de figura para 3x2 subplots
    plt.figure(figsize=(14, 18))

    # Subplot 1: Delta vs S
    plt.subplot(3, 2, 1)
    # USAMOS r'' (Raw String) para las etiquetas Matplotlib con LaTeX
    plt.plot(S_range_greeks, call_delta_range, label=r'Delta Call ($\Delta_C$)', color='darkgreen', linewidth=2)
    plt.plot(S_range_greeks, put_delta_range, label=r'Delta Put ($\Delta_P$)', color='purple', linewidth=2)
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.0, label=f'Strike (K={K_strike:.2f})')
    plt.axhline(y=1.0, color='black', linestyle=':', linewidth=0.5)
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Precio Subyacente (S)', fontsize=10)
    plt.ylabel(r'Delta ($\Delta$)', fontsize=10)
    plt.title(r'Sensibilidad del Delta ($\Delta$) vs S (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='lower right')

    # Subplot 2: Gamma vs S
    plt.subplot(3, 2, 2)
    plt.plot(S_range_greeks, gamma_range, label=r'Gamma ($\Gamma$)', color='darkred', linewidth=2)
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.0, label=f'Strike (K={K_strike:.2f})')
    plt.xlabel('Precio Subyacente (S)', fontsize=10)
    plt.ylabel(r'Gamma ($\Gamma$)', fontsize=10)
    plt.title(r'Sensibilidad del Gamma ($\Gamma$) vs S (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper right')

    # Subplot 3: Vega vs S
    plt.subplot(3, 2, 3)
    plt.plot(S_range_greeks, vega_range, label=r'Vega ($\mathcal{V}$)', color='orange', linewidth=2)
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.0, label=f'Strike (K={K_strike:.2f})')
    plt.xlabel('Precio Subyacente (S)', fontsize=10)
    plt.ylabel(r'Vega ($\mathcal{V}$)', fontsize=10)
    plt.title(r'Sensibilidad del Vega ($\mathcal{V}$) vs S (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper right')

    # Subplot 4: Theta vs S
    plt.subplot(3, 2, 4)
    plt.plot(S_range_greeks, call_theta_range, label=r'Theta Call ($\Theta_C$)', color='darkblue', linewidth=2)
    plt.plot(S_range_greeks, put_theta_range, label=r'Theta Put ($\Theta_P$)', color='teal', linewidth=2, linestyle='--')
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.0, label=f'Strike (K={K_strike:.2f})')
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Precio Subyacente (S)', fontsize=10)
    plt.ylabel(r'Theta por día ($\Theta$)', fontsize=10)
    plt.title(r'Sensibilidad del Theta por día ($\Theta$) vs S (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='lower left')

    # Subplot 5: Rho vs S
    plt.subplot(3, 2, 5)
    plt.plot(S_range_greeks, call_rho_range, label=r'Rho Call ($\rho_C$)', color='brown', linewidth=2)
    plt.plot(S_range_greeks, put_rho_range, label=r'Rho Put ($\rho_P$)', color='red', linewidth=2, linestyle='--')
    plt.axvline(x=K_strike, color='gray', linestyle='--', linewidth=1.0, label=f'Strike (K={K_strike:.2f})')
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Precio Subyacente (S)', fontsize=10)
    plt.ylabel(r'Rho ($\rho$)', fontsize=10)
    plt.title(r'Sensibilidad del Rho ($\rho$) vs S (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

    # ---------------------------------------------------------
    # --- 6. Graficar Griegas (Delta, Gamma, Theta, Vega, Rho) vs T ---
    # ---------------------------------------------------------

    print("\n--- Generando Gráficos de Griegas (Delta, Gamma, Theta, Vega, Rho vs Tiempo a Vencimiento) ---")

    # Rango de tiempo a vencimiento para graficar (de casi 0 a T_maturity * 1.5, por ejemplo)
    # Evitar T=0 para el cálculo
    T_range_greeks = np.linspace(0.01, T_maturity * 1.5, 100)

    # Crear una instancia del modelo con los parámetros de referencia para calcular griegas vs T
    # Usamos el S actual y la volatilidad histórica
    bsm_T = BlackScholesModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma_vol)

    # Calcular Griegas a través del rango T_range_greeks
    call_delta_T = bsm_T.delta_vs_T(T_range_greeks, option_type='call')
    put_delta_T = bsm_T.delta_vs_T(T_range_greeks, option_type='put')
    gamma_T = bsm_T.gamma_vs_T(T_range_greeks)
    theta_T = bsm_T.theta_vs_T(T_range_greeks, option_type='call') # Theta Call vs T
    vega_T = bsm_T.vega_vs_T(T_range_greeks)
    call_rho_T = bsm_T.rho_vs_T(T_range_greeks, option_type='call') # Rho Call vs T
    put_rho_T = bsm_T.rho_vs_T(T_range_greeks, option_type='put') # Rho Put vs T


    # Ajustar el rango de T para que coincida con las griegas calculadas (ignorando T=0)
    T_range_greeks_clean = [t for t in T_range_greeks if t > 0]


    # Tamaño de figura para 3x2 subplots
    plt.figure(figsize=(14, 18))

    # Subplot 1: Delta vs T
    plt.subplot(3, 2, 1)
    plt.plot(T_range_greeks_clean, call_delta_T, label=r'Delta Call ($\Delta_C$)', color='darkgreen', linewidth=2)
    plt.plot(T_range_greeks_clean, put_delta_T, label=r'Delta Put ($\Delta_P$)', color='purple', linewidth=2)
    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.axhline(y=1.0, color='black', linestyle=':', linewidth=0.5)
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=10)
    plt.ylabel(r'Delta ($\Delta$)', fontsize=10)
    plt.title(r'Sensibilidad del Delta ($\Delta$) vs Tiempo (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper right')

    # Subplot 2: Gamma vs T
    plt.subplot(3, 2, 2)
    plt.plot(T_range_greeks_clean, gamma_T, label=r'Gamma ($\Gamma$)', color='darkred', linewidth=2)
    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=10)
    plt.ylabel(r'Gamma ($\Gamma$)', fontsize=10)
    plt.title(r'Sensibilidad del Gamma ($\Gamma$) vs Tiempo (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper right')

    # Subplot 3: Vega vs T
    plt.subplot(3, 2, 3)
    plt.plot(T_range_greeks_clean, vega_T, label=r'Vega ($\mathcal{V}$)', color='orange', linewidth=2)
    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=10)
    plt.ylabel(r'Vega ($\mathcal{V}$)', fontsize=10)
    plt.title(r'Sensibilidad del Vega ($\mathcal{V}$) vs Tiempo (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper right')

    # Subplot 4: Theta vs T
    plt.subplot(3, 2, 4)
    plt.plot(T_range_greeks_clean, theta_T, label=r'Theta Call ($\Theta_C$)', color='darkblue', linewidth=2)
    # Si deseas graficar Put Theta vs T, puedes añadirlo aquí
    # put_theta_T = bsm_T.theta_vs_T(T_range_plot, option_type='put')
    # plt.plot(T_range_plot_clean, put_theta_T, label=r'Theta Put ($\Theta_P$)', color='teal', linewidth=2, linestyle='--')

    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=10)
    plt.ylabel(r'Theta por día ($\Theta$)', fontsize=10)
    plt.title(r'Sensibilidad del Theta por día ($\Theta$) vs Tiempo (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='lower left')

    # Subplot 5: Rho vs T
    plt.subplot(3, 2, 5)
    plt.plot(T_range_greeks_clean, call_rho_T, label=r'Rho Call ($\rho_C$)', color='brown', linewidth=2)
    plt.plot(T_range_greeks_clean, put_rho_T, label=r'Rho Put ($\rho_P$)', color='red', linewidth=2, linestyle='--')
    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.axhline(y=0.0, color='black', linestyle=':', linewidth=0.5)
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=10)
    plt.ylabel(r'Rho ($\rho$)', fontsize=10)
    plt.title(r'Sensibilidad del Rho ($\rho$) vs Tiempo (Vol Hist: ' + f'{sigma_vol*100:.2f}%)', fontsize=12)
    plt.grid(True, linestyle='--')
    plt.legend(loc='upper left')

    plt.tight_layout()
    plt.show()


    # ---------------------------------------------------------
    # --- 7. Graficar Volatilidad Implícita (IV Skew) ---
    # ---------------------------------------------------------
    # Esta sección ahora usa current_S y T_maturity obtenidos al inicio

    if EXPIRY_DATE in yf.Ticker(TICKER).options: # Check again if expiry is available
        print("\n--- 7. Generando Curva de Volatilidad Implícita (IV Skew) ---")

        try:
            ticker_yf = yf.Ticker(TICKER) # Re-instantiate in case of previous error
            options_chain = ticker_yf.option_chain(EXPIRY_DATE)
            calls_df = options_chain.calls
            puts_df = option_chain.puts

            # 7.1. Obtener Strikes y Precios de Mercado
            # Unimos Call y Put dataframes y limpiamos datos
            options_df = calls_df.merge(puts_df, on='strike', suffixes=('_call', '_put'))
            options_df = options_df.dropna(subset=['bid_call', 'ask_call', 'bid_put', 'ask_put'])

            # Calculamos el precio medio de mercado (midPrice)
            options_df['mid_price_call'] = (options_df['bid_call'] + options_df['ask_call']) / 2.0
            options_df['mid_price_put'] = (options_df['bid_put'] + options_df['ask_put']) / 2.0

            # Filtramos opciones con Mid Price cero o volumen/interés abierto muy bajo (opcional, para mayor limpieza)
            options_df = options_df[
                (options_df['mid_price_call'] > 0.001) &
                (options_df['mid_price_put'] > 0.001)
            ]

            # 7.2. Calcular IV para todos los Strikes

            iv_results = []
            for index, row in options_df.iterrows():
                strike = row['strike']

                # Calcular IV Call
                iv_call = calculate_implied_volatility(
                    market_price=row['mid_price_call'],
                    S=current_S, K=strike, T=T_maturity, r=r_rate, option_type='call'
                )

                # Calcular IV Put
                iv_put = calculate_implied_volatility(
                    market_price=row['mid_price_put'],
                    S=current_S, K=strike, T=T_maturity, r=r_rate, option_type='put'
                )

                iv_results.append({
                    'strike': strike,
                    'iv_call': iv_call * 100 if not np.isnan(iv_call) else np.nan,
                    'iv_put': iv_put * 100 if not np.isnan(iv_put) else np.nan
                })

            iv_df = options_df.merge(
                (
                    options_df[['strike']].assign(
                        iv_call=[r['iv_call'] for r in iv_results],
                        iv_put=[r['iv_put'] for r in iv_results]
                    )
                ),
                on='strike', how='left'
            )

            # Limpiamos resultados NaN para graficar
            iv_df_clean = iv_df.dropna(subset=['iv_call', 'iv_put'])

            # 7.3. Graficar la Curva de Volatilidad

            plt.figure(figsize=(10, 6))

            plt.scatter(iv_df_clean['strike'], iv_df_clean['iv_call'], label='IV Call', color='blue', s=20)
            plt.scatter(iv_df_clean['strike'], iv_df_clean['iv_put'], label='IV Put', color='red', s=20, marker='x')

            # Línea de referencia del precio spot actual
            plt.axvline(x=current_S, color='green', linestyle=':', linewidth=1.5, label=f'Spot Price (S={current_S:.2f})')

            plt.xlabel('Precio de Ejercicio (Strike K)', fontsize=12)
            plt.ylabel('Volatilidad Implícita (IV %)', fontsize=12)
            plt.title(f'Curva de Volatilidad Implícita (IV Skew) para {TICKER} - {EXPIRY_DATE}', fontsize=14, fontweight='bold')
            plt.grid(True, linestyle='--')
            plt.legend()
            plt.show()

        except Exception as e:
            print(f"\nError al procesar datos de opciones para IV Skew: {e}")
            iv_df_clean = None # Asegurarse de que iv_df_clean esté definido como None en caso de error

    elif EXPIRY_DATE in yf.Ticker(TICKER).options:
         # Esto puede ocurrir if se encontraron strikes pero todos tenían precios inválidos/NaN para IV
         print(f"\nAdvertencia: No hay datos de IV válidos para graficar el IV Skew para {TICKER} - {EXPIRY_DATE}.")
    else:
        print(f"\nSaltando la generación del gráfico de IV Skew ya que la fecha de vencimiento {EXPIRY_DATE} no está disponible.")


    # ---------------------------------------------------------
    # --- 8. Visualizar Comparación de Precios (BSM vs Binomial) vs. Volatilidad ---
    # ---------------------------------------------------------
    print("\n--- Generando Gráfico de Comparación (Precio BSM vs Binomial vs Volatilidad) ---")
    sigma_range_comp = np.linspace(0.01, sigma_vol * 2.0, 100)
    bsm_call_prices_sigma_comp = [BlackScholesModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma).call_price() for sigma in sigma_range_comp]
    bsm_put_prices_sigma_comp = [BlackScholesModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma).put_price() for sigma in sigma_range_comp]
    binomial_call_prices_sigma_comp = [BinomialModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma, n=n_steps_binomial).call_price() for sigma in sigma_range_comp]
    binomial_put_prices_sigma_comp = [BinomialModel(S=current_S, K=K_strike, T=T_maturity, r=r_rate, sigma=sigma, n=n_steps_binomial).put_price() for sigma in sigma_range_comp]

    plt.figure(figsize=(12, 7))
    plt.plot(sigma_range_comp, bsm_call_prices_sigma_comp, label='Black-Scholes Call', color='blue', linestyle='-', linewidth=2)
    plt.plot(sigma_range_comp, binomial_call_prices_sigma_comp, label=f'Binomial Call (n={n_steps_binomial})', color='cyan', linestyle='--', linewidth=2)
    plt.plot(sigma_range_comp, bsm_put_prices_sigma_comp, label='Black-Scholes Put', color='red', linestyle='-', linewidth=2)
    plt.plot(sigma_range_comp, binomial_put_prices_sigma_comp, label=f'Binomial Put (n={n_steps_binomial})', color='orange', linestyle='--', linewidth=2)
    plt.axvline(x=sigma_vol, color='gray', linestyle='--', linewidth=1.5, label=f'Volatilidad Histórica ($\\sigma_{{hist}}$={sigma_vol*100:.2f}%)')
    plt.xlabel('Volatilidad ($\\sigma$)', fontsize=12)
    plt.ylabel('Precio de la Opción', fontsize=12)
    plt.title(r'Comparación de Precios de Opciones (BSM vs Binomial) vs. Volatilidad' + f'\n(S={current_S:.2f}, K={K_strike:.2f}, T={T_maturity} años, r={r_rate:.2%}, n={n_steps_binomial})', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, linestyle='--')
    plt.show()

    # ---------------------------------------------------------
    # --- 9. Visualizar Comparación de Precios (BSM vs Binomial) vs. Tiempo a Vencimiento ---
    # ---------------------------------------------------------
    print("\n--- Generando Gráfico de Comparación (Precio BSM vs Binomial vs Tiempo a Vencimiento) ---")
    T_range_comp = np.linspace(0.01, T_maturity * 2.0, 100)
    bsm_call_prices_T_comp = [BlackScholesModel(S=current_S, K=K_strike, T=t, r=r_rate, sigma=sigma_vol).call_price() for t in T_range_comp]
    bsm_put_prices_T_comp = [BlackScholesModel(S=current_S, K=K_strike, T=t, r=r_rate, sigma=sigma_vol).put_price() for t in T_range_comp]
    binomial_call_prices_T_comp = []
    binomial_put_prices_T_comp = []
    for t in T_range_comp:
        if t > 0:
            binomial_model = BinomialModel(S=current_S, K=K_strike, T=t, r=r_rate, sigma=sigma_vol, n=n_steps_binomial)
            binomial_call_prices_T_comp.append(binomial_model.call_price())
            binomial_put_prices_T_comp.append(binomial_model.put_price())
        else:
            binomial_call_prices_T_comp.append(np.nan)
            binomial_put_prices_T_comp.append(np.nan)


    plt.figure(figsize=(12, 7))
    plt.plot(T_range_comp, bsm_call_prices_T_comp, label='Black-Scholes Call', color='blue', linestyle='-', linewidth=2)
    plt.plot(T_range_comp, binomial_call_prices_T_comp, label=f'Binomial Call (n={n_steps_binomial})', color='cyan', linestyle='--', linewidth=2)
    plt.plot(T_range_comp, binomial_put_prices_T_comp, label='Black-Scholes Put', color='red', linestyle='-', linewidth=2)
    plt.plot(T_range_comp, binomial_put_prices_T_comp, label=f'Binomial Put (n={n_steps_binomial})', color='orange', linestyle='--', linewidth=2)
    plt.axvline(x=T_maturity, color='gray', linestyle='--', linewidth=1.0, label=f'T Inicial ({T_maturity:.2f} años)')
    plt.xlabel('Tiempo a Vencimiento (Años)', fontsize=12)
    plt.ylabel('Precio de la Opción', fontsize=12)
    plt.title(r'Comparación de Precios de Opciones (BSM vs Binomial) vs. Tiempo a Vencimiento' + f'\n(S={current_S:.2f}, K={K_strike:.2f}, r={r_rate:.2%}, $\sigma$={sigma_vol*100:.2f}%, n={n_steps_binomial})', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, linestyle='--')
    plt.show()


    print("\n---------------------------------------------------------")
    print("Fin del análisis y comparación de modelos.")

<>:550: SyntaxWarning: invalid escape sequence '\s'
<>:889: SyntaxWarning: invalid escape sequence '\s'
<>:550: SyntaxWarning: invalid escape sequence '\s'
<>:889: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3266967029.py:550: SyntaxWarning: invalid escape sequence '\s'
  plt.title(r'Comparación de Precios de Opciones (BSM vs Binomial) vs. Precio Subyacente' + f'\n(T={T_maturity} años, r={r_rate:.2%}, $\sigma$={sigma_vol*100:.2f}%)', fontsize=14, fontweight='bold')
/tmp/ipython-input-3266967029.py:889: SyntaxWarning: invalid escape sequence '\s'
  plt.title(r'Comparación de Precios de Opciones (BSM vs Binomial) vs. Tiempo a Vencimiento' + f'\n(S={current_S:.2f}, K={K_strike:.2f}, r={r_rate:.2%}, $\sigma$={sigma_vol*100:.2f}%, n={n_steps_binomial})', fontsize=14, fontweight='bold')


---------------------------------------------------------
   Cálculo del Precio de Opciones y Griegas (Black-Scholes) y Comparación con Modelo Binomial
---------------------------------------------------------

--- Obtención de Datos de Mercado y Volatilidad para AAPL ---
  S Actual de AAPL: $253.87
  Volatilidad Histórica Anualizada (1y): 32.3419%

--- Análisis de Mercado para AAPL (Strike=105.00, Vencimiento=2025-12-19) ---
  Strike y Vencimiento: K=$105.00, T=0.5 años

--- Comparación de Precios (usando Volatilidad Histórica de 32.34%) ---
| Tipo | Mid Price | Teórico | Desviación |
|------|-----------|---------|------------|
| Call | $149.2750 | $151.4628 | -2.1878 |
| Put  | $0.0350 | $0.0003 | 0.0347 |

--- Volatilidad Implícita (IV) (La 'sigma' que usa el mercado) ---
IV Put: 45.3443%

--- Comparando Precios Teóricos (BSM vs Binomial - Punto Específico) ---
Parámetros: S=253.87, K=105.00, T=0.5, r=0.0500, sigma=0.3234, n=500
Precio Teórico Call (Black-Scholes): $151.4628
Precio 